In [51]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.api import OLS, add_constant
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from statsmodels.formula.api import logit
from patsy import dmatrices
import statsmodels.api as sm




In [111]:
df = {
    'price' : [1000,2000, 3000, 4000, 5000],
    'sqft' : [100,150,200,250,300],
    'bhk' : [1,2,3,4,5],
    'shop' : [1,2,3,4,5],
    'kmtoAir' : [50,40,30,20,10],
    'buy' : [0,1,0,1,1]
}
df = pd.DataFrame(df)

In [61]:

df['sqft'].shape[0]
X = df[['sqft', 'bhk', 'shop', 'bhk', 'shop', 'kmtoAir']]
y = df['price']
X.shape
y.shape

(5,)

In [62]:
def linear_model_diagnostics(X, y):
    #print("\nLinear Regression Diagnostics:")
    #n_features = X.shape[1]
    #correlated_features = int((3 * n_features) // 4)
    #print(f"Assuming at least {(3 * n_features) // 4} features are correlated with target.")

    # VIF Check
    X_const = sm.add_constant(X)
    print(X_const)
    vif = pd.DataFrame()
    vif["feature"] = X_const.columns
    vif["VIF"] = [variance_inflation_factor(X_const.values, i) for i in range(X_const.shape[1])]
    print("\nVariance Inflation Factor (VIF):")
    print(vif)

    # Breusch-Pagan Test
    model = OLS(y, X_const).fit()
    _, pval, _, f_pval = het_breuschpagan(model.resid, X_const)
    print(f"Breusch-Pagan p-value: {pval:.4f}(p > 0.05 suggests homoscedasticity)")

    # Outlier detection with IQR
    print("IQR Analysis for Outliers:")
    Q1 = X.quantile(0.25)
    Q3 = X.quantile(0.75)
    IQR = Q3 - Q1
    outliers = ((X < (Q1 - 1.5 * IQR)) | (X > (Q3 + 1.5 * IQR))).sum()
    print(outliers)

    return model.summary()

In [63]:
linear_model_diagnostics(X,y)

   const  sqft  bhk  shop  bhk  shop  kmtoAir
0    1.0   100    1     1    1     1       50
1    1.0   150    2     2    2     2       40
2    1.0   200    3     3    3     3       30
3    1.0   250    4     4    4     4       20
4    1.0   300    5     5    5     5       10

Variance Inflation Factor (VIF):
   feature  VIF
0    const  0.0
1     sqft  inf
2      bhk  inf
3     shop  inf
4      bhk  inf
5     shop  inf
6  kmtoAir  inf
Breusch-Pagan p-value: 0.9707(p > 0.05 suggests homoscedasticity)
IQR Analysis for Outliers:
sqft       0
bhk        0
shop       0
bhk        0
shop       0
kmtoAir    0
dtype: int64


c:\Users\abhis\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1783: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
c:\Users\abhis\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
c:\Users\abhis\anaconda3\Lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.716e+30
Date:                Mon, 21 Apr 2025   Prob (F-statistic):           9.81e-46
Time:                        19:41:57   Log-Likelihood:                 127.93
No. Observations:                   5   AIC:                            -251.9
Df Residuals:                       3   BIC:                            -252.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1548   6.46e-16   -2.4e+14      0.000      -0.155      -0.155
sqft          17.1092    7.2e-15   2.38e+15      0.000      17.109      17.109
bhk            0.4969   7.53e-16    6.6e+14      0.000       0.497       0.497
shop           0.4969   7.53e-16    6.6e+14      0.000       0.497       0.497
bhk            0.4969   7.53e-16    6.6e+14      0.000       0.497       0.497
shop           0.4969   7.53e-16    6.6e+14      0.000       0.497       0.497
kmtoAir      -14.2551   4.62e-14  -3.08e+14      0.000     -14.255     -14.255
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   0.184
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.615
Skew:                           0.804   Prob(JB):                        0.735
Kurtosis:                       2.395   Cond. No.                     1.11e+32
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The input rank is higher than the number of observations.
[3] The smallest eigenvalue is 1.86e-59. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan

# Sample dataset
df = {
    'price': [1000, 2000, 3000, 4000, 5000],
    'sqft': [100, 150, 200, 250, 300]}
df = pd.DataFrame(df)
X = df[['sqft']]
y = df['price']

def linear_model_diagnostics(X, y):
    print("\n--- Linear Regression Diagnostics ---")

    # VIF Check
    X_const = sm.add_constant(X)
    print(X_const)
    vif = pd.DataFrame()
    vif["feature"] = X_const.columns
    vif["VIF"] = [variance_inflation_factor(X_const.values, i) for i in range(X_const.shape[1])]
    print("\nVariance Inflation Factor (VIF):")
    print(vif)

    # Breusch-Pagan Test
    model = sm.OLS(y, X_const).fit()
    _, pval, _, _ = het_breuschpagan(model.resid, X_const)
    print(f"\nBreusch-Pagan p-value: {pval:.4f} (p > 0.05 suggests homoscedasticity)")

    # IQR for outliers
    print("\nIQR Analysis:")
    Q1 = X.quantile(0.25)
    Q3 = X.quantile(0.75)
    IQR = Q3 - Q1
    outliers = ((X < (Q1 - 1.5 * IQR)) | (X > (Q3 + 1.5 * IQR))).sum()
    print(f"Outliers per feature:\n{outliers}")

    print("\nModel Summary:")
    print(model.summary())

def polynomial_regression(X, y, degree=2):
    print(f"\n--- Polynomial Regression (degree={degree}) ---")
    poly = PolynomialFeatures(degree)
    X_poly = poly.fit_transform(X)
    model = LinearRegression().fit(X_poly, y)
    y_pred = model.predict(X_poly)
    mse = mean_squared_error(y, y_pred)
    print(f"MSE: {mse:.4f}")
    print(f"Coefficients: {model.coef_}")
    print(f"Intercept: {model.intercept_}")

# Run diagnostics
linear_model_diagnostics(X, y)
polynomial_regression(X, y, degree=2)



--- Linear Regression Diagnostics ---
   const  sqft
0    1.0   100
1    1.0   150
2    1.0   200
3    1.0   250
4    1.0   300
Empty DataFrame
Columns: []
Index: []

Variance Inflation Factor (VIF):
  feature  VIF
0   const  9.0
1    sqft  1.0

Breusch-Pagan p-value: 0.1138 (p > 0.05 suggests homoscedasticity)

IQR Analysis:
Outliers per feature:
sqft    0
dtype: int64

Model Summary:
                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.768e+30
Date:                Mon, 21 Apr 2025   Prob (F-statistic):           9.38e-46
Time:                        18:57:18   Log-Likelihood:                 128.01
No. Observations:                   5   AIC:                            -252.0
Df Residuals:                       3   BIC:             

c:\Users\abhis\anaconda3\Lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [64]:
def polynomial_regression(X, y, degree=2):
    print(f"\nPolynomial Regression (degree={degree})")
    poly = PolynomialFeatures(degree)
    X_poly = poly.fit_transform(X)
    model = LinearRegression().fit(X_poly, y)
    predictions = model.predict(X_poly)
    mse = mean_squared_error(y, predictions)
    print(f"MSE: {mse:.4f}")
    return model

In [65]:
polynomial_regression(X,y)


Polynomial Regression (degree=2)
MSE: 0.0000


LinearRegression()

In [99]:
def logistic_regression_diagnostics(X, y):
    print("\n--- Logistic Regression Diagnostics ---")

    if len(np.unique(y)) != 2:
        raise ValueError("Target is not binary.")

    if len(y) <= 1000:
        print("Warning: Less than 1000 samples.")

    # Class balance
    class_counts = np.bincount(y)
    balance_ratio = min(class_counts) / max(class_counts)
    print(f"Class balance ratio: {balance_ratio:.2f}")

    # Box-Tidwell test
    print("\nBox-Tidwell Test (Linearity between features & log odds):")
    df = pd.DataFrame(X.copy())
    df['target'] = y

    for col in df.columns[:-1]:
        if (df[col] <= 0).any():
            print(f"Skipping {col} due to non-positive values (required for log).")
            continue
        df[f"{col}_log"] = df[col] * np.log(df[col])

    # Build formula
    predictors = [col for col in df.columns if col != 'target']
    formula = "target ~ " + " + ".join(predictors)
    
    y_, X_ = dmatrices(formula, df, return_type='dataframe')
    model = logit(formula, df).fit(disp=False)
    print(model.summary())

    # Train/Test logistic model
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
    clf = LogisticRegression(max_iter=1000).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"\nAccuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return clf

In [86]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from statsmodels.formula.api import logit
from patsy import dmatrices
import warnings
warnings.filterwarnings("ignore")  # Suppress convergence warnings from statsmodels

def logistic_regression_diagnostics(X, y):
    print("\n--- Logistic Regression Diagnostics ---")

    if len(np.unique(y)) != 2:
        raise ValueError("Target is not binary.")

    if len(y) <= 1000:
        print("Warning: Less than 1000 samples.")

    # Class balance
    class_counts = np.bincount(y)
    balance_ratio = min(class_counts) / max(class_counts)
    print(f"Class balance ratio: {balance_ratio:.2f}")

    # Box-Tidwell test
    print("\nBox-Tidwell Test (Linearity between features & log odds):")
    df = pd.DataFrame(X.copy())
    df['target'] = y

    for col in df.columns[:-1]:
        if (df[col] <= 0).any():
            print(f"Skipping {col} due to non-positive values (required for log).")
            continue
        df[f"{col}_log"] = df[col] * np.log(df[col])

    # Build formula
    predictors = [col for col in df.columns if col != 'target']
    formula = "target ~ " + " + ".join(predictors)
    
    y_, X_ = dmatrices(formula, df, return_type='dataframe')
    model = logit(formula, df).fit(disp=False)
    print(model.summary())

    # Train/Test logistic model
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
    clf = LogisticRegression(max_iter=1000).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"\nAccuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return clf

# Apply on your data
df = { 
    'price' : [1000,2000, 3000, 4000, 5000],
    'bhk' : [1,2,2,8,4],
    'sqft' : [100,150,150,300,250],
    
    'buy' : [0,1,0,1,1]
}
df = pd.DataFrame(df)
X = df.drop(columns='buy')
y = df['buy']

logistic_regression_diagnostics(X, y)



--- Logistic Regression Diagnostics ---
Class balance ratio: 0.67

Box-Tidwell Test (Linearity between features & log odds):
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:                    5
Model:                          Logit   Df Residuals:                        0
Method:                           MLE   Df Model:                            4
Date:                Mon, 21 Apr 2025   Pseudo R-squ.:                   1.000
Time:                        20:02:23   Log-Likelihood:            -5.6712e-12
converged:                      False   LL-Null:                       -3.3651
Covariance Type:            nonrobust   LLR p-value:                    0.1509
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.8150        nan        nan        nan         nan         nan
price

LogisticRegression(max_iter=1000)

In [120]:
X = df.drop(columns=['buy','price'])
y = df['price']

In [116]:
X

,sqft
0,100
1,150
2,200
3,250
4,300


In [117]:
logistic_regression_diagnostics(X,y)


--- Logistic Regression Diagnostics ---
Class balance ratio: 0.67

Box-Tidwell Test (Linearity between features & log odds):
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:                    5
Model:                          Logit   Df Residuals:                        2
Method:                           MLE   Df Model:                            2
Date:                Mon, 21 Apr 2025   Pseudo R-squ.:                  0.2822
Time:                        20:12:46   Log-Likelihood:                -2.4155
converged:                       True   LL-Null:                       -3.3651
Covariance Type:            nonrobust   LLR p-value:                    0.3869
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.4617     20.299     -0.072      0.943     -41.247      38.323
sqft 

LogisticRegression(max_iter=1000)

In [121]:
def regularized_models(X, y):
    print("\nLasso, Ridge, and Elastic Net Comparison:")
    models = {
        "Lasso": Lasso(alpha=0.1, max_iter=10000),
        "Ridge": Ridge(alpha=1.0),
        "ElasticNet": ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=10000)
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    results = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        if len(np.unique(y)) == 2:
            y_pred_class = np.round(y_pred)
            acc = accuracy_score(y_test, y_pred_class)
            f1 = f1_score(y_test, y_pred_class)
        else:
            acc = model.score(X_test, y_test)
            f1 = None
        results[name] = {"accuracy": acc, "f1_score": f1}

    best_model = max(results, key=lambda k: results[k]["accuracy"])
    print(f"Best Model: {best_model}")
    print(results)
    return models[best_model]


In [122]:
regularized_models(X,y)


Lasso, Ridge, and Elastic Net Comparison:
Best Model: Lasso
{'Lasso': {'accuracy': 0.9999999999937551, 'f1_score': None}, 'Ridge': {'accuracy': 0.9999999743825008, 'f1_score': None}, 'ElasticNet': {'accuracy': 0.9999999999608087, 'f1_score': None}}


Lasso(alpha=0.1, max_iter=10000)

In [ ]:
# df = pd.read_csv("data.csv")
# X = df.drop("target", axis=1)
# y = df["target"]

# linear_model_diagnostics(X, y)
# polynomial_regression(X, y)
# logistic_regression_diagnostics(X, y)
# regularized_models(X, y)